In [1]:
import numpy as np
import math
import cv2
import numpy

In [2]:
def rotationMatrixToEulerAngles(R):

    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

In [3]:
class Camera_config:
    def __init__(self, intrinsic_yaml_file, extrinsic_yaml_file, name="camera_name"):

        self.camera_name = name
        intrinsic_yaml = cv2.FileStorage(intrinsic_yaml_file, cv2.FILE_STORAGE_READ)
        extrinsic_yaml = cv2.FileStorage(extrinsic_yaml_file, cv2.FILE_STORAGE_READ)

        # https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
        # https://www.geeksforgeeks.org/calibratecamera-opencv-in-python/

        # 9 items: Camera matrix
        self.in_K = intrinsic_yaml.getNode("K").mat()
        # 5 items: Distortion coefficients
        self.in_D = intrinsic_yaml.getNode("D").mat()

        self.in_xi = intrinsic_yaml.getNode("xi").mat()  # 0

        # 3 items
        in_image_shape = intrinsic_yaml.getNode("image_shape").mat()

        # 9 items: rotation vectors
        self.ex_rot_mat = extrinsic_yaml.getNode("rot_mat").mat()

        # 3 items:  translation vectors
        self.ex_t_vec = extrinsic_yaml.getNode("t_vec").mat()

        # 12 items:
        self.ex_camera_matrix_p = extrinsic_yaml.getNode("camera_matrix_p").mat()

        # 3 items: xyz position
        self.ex_position = extrinsic_yaml.getNode("position").mat()

        # http://sdformat.org/tutorials?tut=specify_pose
        # The elements x y z specify the position vector (in meters), and the elements roll pitch yaw are Euler angles (in radians) that specify the orientation, which can be computed by an extrinsic X-Y-Z rotation

        self.x_ = self.ex_position[0][0]
        self.y_ = self.ex_position[1][0]
        self.z_ = self.ex_position[2][0]

        invR = numpy.linalg.inv(self.ex_rot_mat)
        neg_invR = numpy.negative(invR)
        neg_invR_t = numpy.dot(neg_invR, self.ex_t_vec)
        self.x = neg_invR_t[0][0]
        self.y = neg_invR_t[1][0]
        self.z = neg_invR_t[2][0]

        # other possible _AXES2TUPLE : https://github.com/matthew-brett/transforms3d/blob/main/transforms3d/euler.py#L148
        # https://www.andre-gaschler.com/rotationconverter/

        self.height = int(in_image_shape[0][0])
        self.width = int(in_image_shape[1][0])

        # 84 degree to 1.46607657 radians
        # http://sdformat.org/spec?ver=1.11&elem=sensor#camera_distortion

        self.k1 = self.in_D[0][0]
        self.k2 = self.in_D[1][0]
        self.p1 = self.in_D[2][0]
        self.p2 = self.in_D[3][0]
        self.k3 = self.in_D[4][0]

        self.px = self.in_K[0][2]
        self.py = self.in_K[1][2]
        self.fx = self.in_K[0][0]
        self.fy = self.in_K[1][1]
        self.fov_x = 2 * math.atan(self.width / (2 * self.fx))
        self.fov_y = 2 * math.atan(self.height / (2 * self.fy))
        # The FoV is
        # ~84 degrees horizontal
        # ~53 degrees vertical
        # print(self.fov_x, self.fov_y) # 53.550787735526455 83.92250895589785

        self.fovhor = self.fov_x
        self.fovaspect = self.fov_x / self.fov_y
        # [OpenCV](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) uses five parameters, known as distortion coefficients given by like this: `k1, k2, p1, p2 , k3 # pay attention to the order`

        # rpy, R, Q, Qx, Qy, Qz = cv2.RQDecomp3x3(self.ex_rot_mat)

        # R_temp = np.array([
        #     [0, -1 ,0],
        #     [0, 0, -1],
        #     [1, 0, 0]
        # ])
        R_world_to_cam = self.ex_rot_mat
        R_cam_to_gazebocam = np.array([[ 0.,  0.,  1.],
                                        [-1., 0., 0.],
                                        [0., -1., 0.]])
        R_world_to_gazebocam = np.matmul(R_cam_to_gazebocam, R_world_to_cam)
        R_gazebocam_to_world = np.linalg.inv(R_world_to_gazebocam)
        self.r, self.p, self.w = rotationMatrixToEulerAngles(R_gazebocam_to_world)
        print(
            f"""<xacro:camera number="{self.camera_name[:-5]}" x="{self.x}" y="{self.y}" z="{self.z}" r="{self.r}" p="{self.p}" w="{self.w}" width="{int(self.width)}" height="{int(self.height)}" k1="{self.k1}" k2="{self.k2}" k3="{self.k3}" p1="{self.p1}" p2="{self.p2}" horizental_fov="{self.fovhor}" aspect_ratio="{self.fovaspect}"  />   """
        )
        for p1,p2 in zip(self.ex_position, [self.x, self.y, self.z]):
            assert np.linalg.norm(p1-p2)<0.000001, "not correct position"


In [4]:

cameras = range(160,172)
for cam in cameras:
    name = f"{cam}.yaml"

    intrinsic_yaml_file = "/home/gpss1/remote/phd/code/bev/mmsegmentation/data/calibrations/intrinsic/" + name
    extrinsic_yaml_file = "/home/gpss1/remote/phd/code/bev/mmsegmentation/data/calibrations/extrinsic/" + name
    Camera_config(intrinsic_yaml_file, extrinsic_yaml_file, name)


<xacro:camera number="160" x="-23.768993347281643" y="1.0959115856381247" z="7.792710237230576" r="1.421493857805801" p="1.0967432285226815" w="-0.135882813463627" width="3840" height="2160" k1="-0.3990864717265948" k2="0.2016451846353012" k3="-0.053753317426588856" p1="0.0003788431220127677" p2="-0.00013457492654973615" horizental_fov="1.4269612320879095" aspect_ratio="1.57319517984556"  />   
<xacro:camera number="161" x="-15.548574448619283" y="1.1875803973838794" z="8.430721825553768" r="1.5097953218375135" p="1.2948111530790358" w="0.005212075916293576" width="3840" height="2160" k1="-0.5337684959685228" k2="0.34369206124627594" k3="-0.11312322233867529" p1="0.0012933658245218613" p2="-0.001568761626559521" horizental_fov="1.2862854346420696" aspect_ratio="1.617131790271812"  />   
<xacro:camera number="162" x="-5.583158189893618" y="1.1215696998181661" z="7.334558192411434" r="1.5153065411297053" p="1.2585498082517528" w="-0.0623840194833805" width="3840" height="2160" k1="-0.388